# Introduction to Models and Reachability in Kaa

We first import the necessary modules and classes from Kaa. To start defining models, we at least need the sympy and numpy packages alongside the Model and Bundle classes.

In [ ]:
import sympy as sp
import numpy as np

from kaa.bundle import Bundle
from kaa.model import Model

# Basic Parallelotope 

A model file in Kaa consists of first defining the variables and the dynamics. Kaa is designed for the analysis of **polynomial discrete dynamics**. The following model definition illustates how to define the unitbox shifting diagonally up to the right.

In [ ]:
class Basic(Model):

    def __init__(self):

        x,y = sp.Symbol('x'), sp.Symbol('y')

        dx = x + 1
        dy = y + 1

        dyns  = [dx, dy]
        vars = [x, y]

        L = np.empty([2,2])
        T = np.empty(2)

        L[0] = [1, 0]
        L[1] = [0, 1]

        T[0][0] = 0
        T[0][1] = 1

        offu = np.empty(2)
        offl = np.empty(2)

        offu[0] = 1
        offu[1] = 1
    
        offl[0] = 1
        offl[1] = 1
  
        b = Bundle(T, L, offu, offl, vars)

        super().__init__(b, dyns, vars)


The numpy matrix $L$ represents the directions matrix of the model. Each row of the matrix reprsents a normal vector to the plane used to define the positive halfspace i.e the $i^{th}$ row of $L$ defines the halfspaces $L_i{\bf x} \leq \text{offu}_i$ and $-L_i{\bf x} \leq \text{offl}_i$ where $\text{offu}$ is the vector of offsets for the upper facets (corresponding to the $L_i$) and $\text{offl}$ is the vector of offsets for the lower facets (corresponding to the $-L_i$).

The matrix $T$ represents the template matrix defining the parallelotope bundle of the model. Each row $T_i$ represents a vector of row-indices in $L$ defining parallelotope $i$ in the bundle. This defines $n$ hyperplanes and their parallel counterparts from the directions matrix. In other words, element $T_{ij}$ indicates that the postive and negative halfspaces defined by normal vector $L_i, -L_i$ with offsets in $\text{offu}_i$ and $\text{offl}_i$ respectively is contained in parallelotope $j$. The intersection of the positive and negative halfspaces will be the parallelotope in question. Note that the number of columns of $T$ must exactly be the dimension of the system. 

Thus, observe that definitions of $L$, $T$, $\text{offu}$. and $\text{offl}$ designate the initial set as the **unit box** in $\mathbb{R}^2$. To initialize the reachability computation, we must first create an initial bundle denoted by the variable $b$ in this case. The bundle is a data structure which contains the directions matrix $L$ along with the upper and lower offsets $\text{offu}$, $\text{offl}$ and the template matrix $T$. It additionally encapsulates methods describing the parallelotope bundle in terms of linear constraints (half-spaces).

In this case, there is only one row in the template matrix. Examples below will leverage multiple parallelotopes to generate tighter over-approximations.

We now construct a Reach object and a FlowPipePlotter object to calculate the reachable set and plot the projection to the $x$-axis for 150 steps.

First, we import the necessary modules and classes as usual:

In [ ]:
from kaa.reach import ReachSet
from kaa.flowpipe import FlowPipePlotter

kaa.reach.ReachSet is the object responsible for computing the reachale set for a supplied number of time steps. The constructor takes in a Model object defining the dynamics and the initial bundle. Calling **ReachSet.computeReachSet(num)** will calculate the reachable set for $num$ time steps. To plot the projection on the $x$-axis, we first create a **FlowPipePlotter** object supplied with the **Flowpipe** returned by **ReachSet.computeReachSet(num)** and feed it into **FlowPipePlotter.plot2DProj(var)** where $var$ is the index of the desired variable. The relevant code is posted below: 

In [ ]:
basic_model = Basic()
basic_reach = ReachSet(basic_model)
basic_flowpipe = basic_reach.computeReachSet(150)
FlowPipePlotter(basic_flowpipe).plot2DProj(0)

We can also plot the phase diagram for this model as follows. This should just be the unit box shifting along the positive diagonal for 150 steps.

## Phase Plot for Basic (x,y)

In [ ]:
basic_model = basic.Basic()
basic_reach = ReachSet(basic_model)
FlowPipePlotter(basic_reach.computeReachSet(150)).plot2DPhase(0,1)

# SIR Epidemic Model

We now investigate a slightly more complicated model, the SIR epidemic model. Following the format from the example above, we define the following model:

In [3]:
class SIR(Model):

  def __init__(self):

      s, i, r = sp.Symbol('s'), sp.Symbol('i'), sp.Symbol('r')

      ds = s - (0.34*s*i)*0.05;
      di = i + (0.34*s*i - 0.05*i)*0.05;
      dr = r + 0.05*i*0.05;

      dyns = [ds, di, dr]
      vars = [s, i, r] 
      sys_dim = len(vars)

      num_direct = 3
      num_temps = 1

      L = np.zeros([num_direct, sys_dim])
      T = np.zeros([num_temps, sys_dim])

      # Directions matrix
      L[0][0] = 1  #[1 0 0 ]^T
      L[1][1] = 1  #[0 1 0 ]^T
      L[2][2] = 1  #[0 0 1 ]^T

      # Template matrix
      T[0][0] = 0 
      T[0][1] = 1 
      T[0][2] = 2

      offu = np.zeros(num_direct)
      offl = np.zeros(num_direct)

      offu[0] = 0.8
      offl[0] = -0.79

      offu[1] = 0.2
      offl[1] = -0.19

      offu[2] = 0.001
      offl[2] = -0.00099

      b = Bundle(T, L, offu, offl, vars)
      super().__init__(b, dyns, vars)


The initial set is taken from Dreossi et. al. Note that we only use one parallelotope here as $T$ only has one row. Furthermore, this parallelotope uses the axis-aligned unit vectors as its directions. Based on the definitions of $\text{offu}$ and $\text{offl}$, observe that initial set will be:
$$ s \in [0.79, 0.8] \quad i \in [0.19, 0.2]  \quad r = 0 $$

We can now calculate the flowpipe for 150 steps, and plot the resulting projections and phase plots. 

## SIR Model projection to r

In [ ]:
sir_model = SIR()
sir_reach = ReachSet(sir_model)
sir_flow = sir_reach.computeReachSet(150)
FlowPipePlotter(sir_flow).plot2DProj(2)

## SIR Model projection to i 

In [ ]:
FlowPipePlotter(sir_flow).plot2DProj(1)

## Phase Plot for SIR (S,I)

In [ ]:
FlowPipePlotter(flowpipe).plot2DPhase(0,1)

## Phase Plot for SIR (I,R)

In [ ]:
FlowPipePlotter(flowpipe).plot2DPhase(1,2)

# Rossler Model

We finally introduce the Rossler model with parameters set by Dreossi et. al as follows:

In [5]:
class Rossler(Model):

    def __init__(self):

        x, y, z = sp.Symbol('x'), sp.Symbol('y'), sp.Symbol('z')
        vars = [x, y, z]

        dim_sys = len(vars)

        dx = x + (-y-z)*0.025
        dy = y + (x + 0.1*y)*0.025
        dz = z + (0.1 + z*(x-14))*0.02

        dyns = [dx, dy ,dz]

        num_direct = 5
        num_temps = 3

        L = np.zeros([num_direct, dim_sys])
        T = np.zeros([num_temps, dim_sys])

        L[0][0] = 1
        L[1][1] = 1
        L[2][2] = 1

        L[3][0] = 1
        L[3][1] = 0.5

        L[4][0] = 0.5
        L[4][2] = 0.5

        T[0][0] = 0; T[0][1] = 1; T[0][2] = 2;
        T[1][0] = 1; T[1][1] = 2; T[1][2] = 3;
        T[2][0] = 2; T[2][1] = 3; T[2][2] = 4;

        offu = np.zeros(num_direct)
        offl = np.zeros(num_direct)

        offu[0] = 0.1; offl[0] = -0.09;
        offu[1] = 5; offl[1] = -4.99;
        offu[2] = 0.1; offl[2] = -0.09;
        offu[3] = 10; offl[3] = 0;
        offu[4] = 10; offl[4] = 0;

        b = Bundle(T, L, offu, offl, vars)

        super().__init__(b, dyns, vars)

## Rossler Model projection to x

In [ ]:
rossler_model = rossler.Rossler()
rossler_reach = ReachSet(rossler_model)
rossler_flow = rossler_reach.computeReachSet(150)
FlowPipePlotter(rossler_flow).plot2DProj(0)

## Rossler Model projection to z

In [ ]:
FlowPipePlotter(rossler_flow).plot2DProj(1)

## Phase Plot for Rossler (x,y)

In [ ]:
FlowPipePlotter(rossler_flow).plot2DPhase(0,1)

## Phase Plot for Rossler (y,z)

In [ ]:
FlowPipePlotter(rossler_flow).plot2DPhase(0,2)

There are other models defined in the **/models** directory. Basic tests are contained in the **/tests** directory. The reader is encouraged to experiment with the values found in these files to learn about reachability with multiple parallelotopes.  

To have a more in-depth look at the theoretical foundations of these techniques, check out the following papers:

[Dang, T., Dreossi, T., Piazza, C.: Parameter synthesis using parallelotopic enclosure and appli- cations to epidemic models](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.707.7012&rep=rep1&type=pdf)

[Dreossi, T., Dang, T., Piazza, C.: Parallelotope bundles for polynomial reachability.](https://dl.acm.org/doi/abs/10.1145/2883817.2883838)

[Dang, T., Testylier, R.: Reachability analysis for polynomial dynamical systems using the bern- stein expansion.](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.300.4012&rep=rep1&type=pdf)